In [ ]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Statistics")
Pkg.add("StatsBase")
Pkg.add("StatsModels")
Pkg.add("GLM")
Pkg.add("CategoricalArrays")

   Resolving package versions...
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Estefany\.julia\envir

In [4]:
using DataFrames
using Statistics
using StatsBase
using CSV
using StatsModels
using GLM

# **Part 1: Double Lasso and DAGs**

### 1. Consider the US census data from the year 2015 to analyse the effect of college graduate (clg) status and it’s interaction effects with gender (sex), location (mw, so,we, ne) and both on wage jointly. All other variables denote some other socio-economic characteristics, e.g. marital status, occupation, and experience.

In [71]:
using Downloads, CSV, DataFrames

# Descargar el archivo localmente
url = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/wage2015_subsample_inference.csv"
local_file = "wage2015_subsample_inference.csv"
Downloads.download(url, local_file)

# Leer el archivo descargado como DataFrame
data = CSV.read(local_file, DataFrame)


Row,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Float64,Float64,Float64,Float64,Float64,Int32,Float64,Int32
1,9.61538,2.26336,1,0,0,0,1,0,0,0,0,1,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,48.0769,3.8728,0,0,0,0,1,0,0,0,0,1,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,11.0577,2.40313,0,0,1,0,0,0,0,0,0,1,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,13.9423,2.63493,1,0,0,0,0,1,0,0,0,1,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,28.8462,3.36198,1,0,0,0,1,0,0,0,0,1,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,11.7308,2.46222,1,0,0,0,1,0,0,0,0,1,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,19.2308,2.95651,1,0,1,0,0,0,0,0,0,1,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,19.2308,2.95651,0,0,1,0,0,0,0,0,0,1,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,12.0,2.48491,1,0,1,0,0,0,0,0,0,1,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


In [44]:
using Pkg
Pkg.add("CategoricalArrays")


   Resolving package versions...
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Manifest.toml`


* Generate the dataset with all the two-way interactions between variables. Make sure that the categorical variables are transformed to dummies properly. Also, note that the resulting dataset contains the treatment and it’s interactions with the other variables of interest, so you don’t need to generate them separately. (2 pts)

In [72]:
using DataFrames, StatsModels


# interacciones bidireccionales
interaction_formula = @formula(lwage ~ 1 + clg + sex + mw + so + we + ne + (clg + sex + mw + so + we + ne )^2)

FormulaTerm
Response:
  lwage(unknown)
Predictors:
  1
  clg(unknown)
  sex(unknown)
  mw(unknown)
  so(unknown)
  we(unknown)
  ne(unknown)
  (clg,sex,mw,so,we,ne)->(clg + sex + mw + so + we + ne) ^ 2

In [73]:
X = modelmatrix(interaction_formula, data);
X = X[:,3:end];
X

5150×6 Matrix{Float64}:
 1.0  0.0  0.0  0.0  1.0  9.0
 0.0  0.0  0.0  0.0  1.0  4.0
 0.0  0.0  0.0  0.0  1.0  1.0
 1.0  0.0  0.0  0.0  1.0  4.0
 1.0  0.0  0.0  0.0  1.0  9.0
 1.0  0.0  0.0  0.0  1.0  9.0
 1.0  0.0  0.0  0.0  1.0  4.0
 0.0  0.0  0.0  0.0  1.0  1.0
 1.0  0.0  0.0  0.0  1.0  4.0
 1.0  0.0  0.0  0.0  1.0  9.0
 ⋮                        ⋮
 0.0  0.0  0.0  1.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0  1.0
 1.0  0.0  0.0  1.0  0.0  9.0
 0.0  0.0  0.0  1.0  0.0  4.0
 0.0  0.0  0.0  1.0  0.0  4.0
 1.0  0.0  0.0  1.0  0.0  4.0
 0.0  0.0  0.0  1.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0  1.0

In [74]:
D = float.(data.clg) 

5150-element Vector{Float64}:
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 1.0
 ⋮
 0.0
 0.0
 1.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0

In [76]:
y_log = data[:, 2]


5150-element Vector{Float64}:
 2.26336437984076
 3.87280229227486
 2.40312632221592
 2.63492793627325
 3.36197666850887
 2.46221523858593
 2.95651156040071
 2.95651156040071
 2.484906649788
 2.95651156040071
 ⋮
 3.11777970799683
 2.82298016777619
 3.17965511171492
 2.62800749342867
 2.69254601456624
 3.13883311719466
 3.64965874096066
 3.4955080611334
 2.85115104474288

In [97]:
train_sample = rand(Float64, size(data)[1]) .< 0.80
test_sample = .!(train_sample)
y_train, y_test = y_log[train_sample], y_log[test_sample]
X_train, X_test = X[train_sample, :], X[test_sample, :];
D_train, D_test = D[train_sample, :], D[test_sample, :];

In [84]:
using Pkg
Pkg.add("GLMNet")

   Resolving package versions...
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Estefany\.julia\environments\v1.10\Manifest.toml`


In [91]:
using GLMNet
using DataFrames

# Ajuste LASSO para Y
model_Y = glmnet(X, y_log, alpha=1.0)

# Ajuste LASSO para D
model_D = glmnet(X, D, alpha=1.0)


Least Squares GLMNet Solution Path (63 solutions for 6 predictors in 385 passes):
───────────────────────────────
      df    pct_dev           λ
───────────────────────────────
 [1]   0  0.0        0.334989
 [2]   1  0.0878989  0.305229
 [3]   1  0.160874   0.278114
 [4]   1  0.221459   0.253407
 [5]   1  0.271758   0.230895
 [6]   1  0.313517   0.210383
 [7]   1  0.348186   0.191693
 [8]   1  0.376969   0.174664
 [9]   1  0.400865   0.159147
[10]   1  0.420704   0.145009
[11]   1  0.437175   0.132127
[12]   2  0.474238   0.120389
[13]   2  0.553292   0.109694
[14]   2  0.618902   0.0999489
[15]   2  0.673372   0.0910697
[16]   2  0.718593   0.0829793
[17]   2  0.756137   0.0756077
[18]   2  0.787252   0.0688909
[19]   2  0.813138   0.0627708
[20]   2  0.83463    0.0571944
[21]   2  0.852473   0.0521134
[22]   2  0.867287   0.0474838
[23]   2  0.879585   0.0432655
[24]   2  0.889796   0.0394219
[25]   2  0.898243   0.0359198
[26]   2  0.905285   0.0327288
[27]   2  0.911132   0.029821

In [99]:
function log_grid(lower::Int, upper::Int, log_step::Float64)
    log_grid = range(lower, stop=upper, length= Int(1 /log_step))
    return exp.(log_grid)
end

log_grid (generic function with 1 method)

In [100]:
function k_folds(data::AbstractArray, k::Int = 5)
    mdl = size(data, 1) % k
    floor = size(data, 1) ÷ k 

    if mdl == 0
        trues = fill(1, floor, 1)
        split_matrix = kron(I(k), trues)
    else
        trues_g1 = fill(1, floor + 1, 1)
        split_matrix_g1 = kron(I(mdl), trues_g1)
        
        trues_g2 = fill(1, floor, 1)
        split_matrix_g2 = kron(I(k - mdl), trues_g2)
        
        split_matrix = [split_matrix_g1  zeros(size(split_matrix_g1, 1), size(split_matrix_g2, 2));
                        zeros(size(split_matrix_g2, 1), size(split_matrix_g1, 2))  split_matrix_g2]
    end
    
    sm_bool = split_matrix .== 1
    splits = [sm_bool[:, x] for x in 1:k]
    
    return splits
end

k_folds (generic function with 2 methods)

In [101]:
using GLMNet
function optimal_lambda(Y::AbstractVector, X::AbstractArray, lambda_bounds::Tuple{Int, Int}, k::Int = 5; niter::Int = 100)
    Y = vec(Y) 

    if ndims(X) == 1
        X = reshape(X, :, 1)
    end

    folds = k_folds(X, k)
    all_lambdas = log_grid(lambda_bounds[1],lambda_bounds[2], 1/niter)
    all_mse = zeros(niter)

    for (j, l) in enumerate(all_lambdas)
        split_pes = zeros(k)
        
        for i in 1:k
            X_train = X[.!folds[i], :]
            X_test = X[folds[i], :]
            y_train = Y[.!folds[i]]
            y_test = Y[folds[i]]

            model = glmnet(X_train, y_train, alpha=1.0, lambda=[l])
            predict = GLMNet.predict(model, X_test)

            pe = sum((y_test - predict).^2)
            split_pes[i] = pe
        end

        all_mse[j] = mean(split_pes)
    end

    selected = argmin(all_mse)
    optimal_lambda = all_lambdas[selected]
    optimal_model = glmnet(X, Y, alpha=1.0, lambda=[optimal_lambda])
    optimal_coef = [optimal_model.a0;optimal_model.betas[:]]

    output = Dict(
        "optimal_lambda" => optimal_lambda,
        "optimal_coef" => optimal_coef,
        "all_lambdas" => all_lambdas,
        "all_mse" => all_mse
    )

    return output
end

optimal_lambda (generic function with 2 methods)

In [102]:
function predict_model(optimal_model::Dict, X::AbstractArray)
    intercept = ones(size(X, 1), 1)
    Z = [intercept;; X]
    
    return Z * optimal_model["optimal_coef"]
end

predict_model (generic function with 1 method)

In [105]:
using GLM, DataFrames

# Fitting a linear regression model
model_ls = lm([ones(size(X_train)[1]);;X_train],y_train)

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int32}}}}:

Coefficients:
─────────────────────────────────────────────────────────────────────────
          Coef.    Std. Error       t  Pr(>|t|)    Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────
x1   2.91635       0.0223415   130.54    <1e-99    2.87255      2.96016
x2  -0.244421      0.025016     -9.77    <1e-21   -0.293466    -0.195376
x3  -0.0779916     0.0255663    -3.05    0.0023   -0.128115    -0.0278678
x4  -0.0293215     0.0247441    -1.18    0.2361   -0.0778334    0.0191903
x5   0.0         NaN           NaN       NaN     NaN          NaN
x6  -0.00423504    0.0264602    -0.16    0.8728   -0.0561113    0.0476412
x7   0.05291       0.00465074   11.38    <1e-28    0.0437921    0.062028
─────────────────────────────────────────────────────────────────────────


In [ ]:
# Finding the optimal lambda and fitting the Lasso model using the training data
model_Y_lasso = optimal_lambda(y_train, X_train, (-7, 7))

# Finding the optimal lambda and fitting the Lasso model using the training data
model_D_lasso = optimal_lambda(D_train, X_train, (-7, 7))